** Neil Garrett, June 2018 **
uses julia EM model fitting by Nathaniel Daw


# Start up commands/load relevant functions

In [1]:

parallel = true # Run on multiple CPUs. If you are having trouble, set parallel = false: easier to debug
full = false    # Maintain full covariance matrix (vs a diagional one) at the group level
emtol = 1e-3    # stopping condition (relative change) for EM

using Distributed
if (parallel)
	# only run this once
	addprocs()
end

# this loads the packages needed -- the @everywhere makes sure they 
# available on all CPUs 

@everywhere using DataFrames
@everywhere using SharedArrays
@everywhere using ForwardDiff
@everywhere using Optim
@everywhere using LinearAlgebra       # for tr, diagonal
@everywhere using StatsFuns           # logsumexp
@everywhere using SpecialFunctions    # for erf
@everywhere using Statistics          # for mean
@everywhere using Distributions
@everywhere using GLM
@everywhere using CSV #for reading/writing csv files

# change this to where you keep the Daw's latest em code
@everywhere directory = "/Users/neil/GitHubRepo/Projects/PreySelection/em"

#load in functions including em
@everywhere include("$directory/em.jl");
@everywhere include("$directory/common.jl");
@everywhere include("$directory/likfuns.jl")


# Data read and process

### Read in trial by trial data

In [2]:

#read in csv file of the data
#trial by trial data: note will include force trials and missed responses
df = CSV.read("/Users/neil/GitHubRepo/Projects/PreySelection/v104/data/trialdata_104_processed.csv");


### Get rid of excluded subs

In [3]:

df = df[df[:exclude].==0,:];


### Convert approach avoid to 2s and 1s , missed as 0. Then convert to integers (necessary to use as an index)

In [4]:

# convert approach_avoid to 1s (avoid) and 2s (approach)
df[df[:approach_avoid].==1,:approach_avoid] = 2
df[df[:approach_avoid].==-1,:approach_avoid] = 1

# put 0 for missed responses
index_NaN = findall(isnan.(df[:approach_avoid]))
df[index_NaN, :approach_avoid] = 0

df[:approach_avoid] = convert(Vector{Integer}, df[:approach_avoid])

first(df, 6)


,subj,trial_index_actual,block,stimulus,stim_rank,reward_percent,delay_s,profitability,stim_left_right,key_press,approach_avoid,rt,rt_z,force_trial,missed,order_condition,exclude,exclude_reason
,Int64⍰,Int64⍰,Int64⍰,String⍰,Int64⍰,Int64⍰,Int64⍰,Float64⍰,String⍰,Int64⍰,Integer,Float64⍰,Float64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,String⍰
1,2,0,1,../static/images/invador1.png,4,20,8,2.5,right,74,2,1253.0,1.13279,0,0,2,0,do not exclude
2,2,1,1,../static/images/invador1.png,4,20,8,2.5,right,-1,0,NaN,NaN,0,1,2,0,do not exclude
3,2,2,1,../static/images/invador3.png,2,20,2,10.0,right,74,2,1185.0,0.859417,0,0,2,0,do not exclude
4,2,3,1,../static/images/invador4.png,1,80,2,40.0,left,70,2,1148.0,0.710668,0,0,2,0,do not exclude
5,2,4,1,../static/images/invador1.png,4,20,8,2.5,right,70,1,1156.0,0.74283,2,0,2,0,do not exclude
6,2,5,1,../static/images/invador1.png,4,20,8,2.5,left,70,0,NaN,NaN,0,1,2,0,do not exclude


# Asymmetric Model

This model comprises: 

1. An intercept which reflects degree of bias to reject.

2. A beta (termperature parameter) which controls sensitivity to the difference between the options (0 = pick 50/50. Higher it is, the more sensative subs are tothe different options (more step functionesque). <br>

3. Two learning rates: one for appetative component (reward), one for aversive (delay)

Uses Q learned average to predict choice

Initalise Qaverage in model at the arithmetic average over all subs over both sessions

In [5]:

@everywhere function model_asymmetric(params, data)
     
    #model parameters
    intercept = params[1]
    beta = params[2]
    lr_pos = 0.5 .+ 0.5.*erf(params[3]/sqrt(2))
    lr_neg = 0.5 .+ 0.5.*erf(params[4]/sqrt(2))
    
    #initalise various variables
    opp_cost_estimate = zeros(typeof(beta),1) # stores estimated opp cost

    #initalise to average rate over the experiment
    Q_estimate = zeros(typeof(beta),1) .+ 8.22 # stores estimated global reward rate
    
    Qd = zeros(typeof(beta),2) # decision variable; 1st element is the opp cost of accepting (or value of rejecting), 2nd element is just the reward of the option (value of accepting)

    lik = 0 #likelihood

    #extract various variables from the dataframe
    reward = data[:reward_percent]
    delay = data[:delay_s]
    force = data[:force_trial]  
    missed = data[:missed] 
    c = data[:approach_avoid]
    sub = data[:subj]
    block = data[:block]
    
    reward_rate_store = [];
        
    for i = 1:length(c)
                    
            # decrease estimate of global reward rate for encounter time (2seconds)
            Q_estimate = (1-lr_neg) * Q_estimate .+ 0
            Q_estimate = (1-lr_neg) * Q_estimate .+ 0
        
            #calculate estimate of opportunity cost given estimate of reward rate and delay incurred by option 
            opp_cost_estimate = Q_estimate*delay[i]
            
            append!(reward_rate_store, Q_estimate)

            # if not a force trial predict choice based on current values
            if ((force[i]<1) & (missed[i]<1))
                        
                # decision variable - the estimate of opportunity cost ("reward" of rejecting) versus 
                # reward of the current option (if accepted)
                Qd = [intercept, 0] .+ [beta.*opp_cost_estimate[1], beta.*reward[i]]

                # increment likelihood
                lik += Qd[c[i]] - log(sum(exp.(Qd)))
            
            end
            
            #incur 8second time out for missed response
            if (missed[i]==1)
            
                for j = 1:8
                
                     Q_estimate = (1-lr_neg) * Q_estimate .+ 0

                end
            
            end
        
            # regardless of whether a force trial or not, 
            # if accept the option, Q_estimate updates and there is a delay incurred
            if ((c[i] == 2) & (missed[i]==0))
                
                for j = 1:delay[i]
                
                    Q_estimate = (1-lr_neg) * Q_estimate .+ 0
                
                end
            
                    Q_estimate = (1-lr_pos) * Q_estimate .+ lr_pos*reward[i]
                
            end
    
    end
        
    # here if running em you can only return the likelihood
    # compile trial by trial values here
    trial_data = DataFrame(sub = sub, block = block, reward_rate = reward_rate_store)
    
    # here if running em you can only return the likelihood
    #return -lik
    
    # but if you run in order to extract trials, subs etc then want to return this
    return (-lik, trial_data)
    
end


# Parameter optimisiation

### load per subject model parameters

In [6]:
# if you already have best fit parameters saved, can read in here (rather than running model to find)
params = CSV.read("subject_params.csv");

In [7]:
subs = unique(df[:subj])

# initalise this - will store all trial to trial parameters
trial_data_compile = []

# run model for each subject using best fit parameters
for x = 1:length(subs)

    # pull out optimal betas for subject - these are used in the model
    # note: you want the unconverted learning score to be fed in
    betas_sub = convert(Array, params[x, [:intercept, :beta, :learning_rate_raw_pos, :learning_rate_raw_neg]])
    data_sub = df[df[:subj].==subs[x], :]
    
    # run model using these parameters - note must have commented in the model to return all of these variables (and not only -lik)
    (minus_li, trial_data) = model_asymmetric(betas_sub, data_sub)
    
    if x==1
        
        trial_data_compile = trial_data
        
    else
        
        append!(trial_data_compile, trial_data)
        
    end
 
end

# check thesne are all the same sizes
print(size(df))
print(size(trial_data_compile))

CSV.write("reward_rate_extract.csv", DataFrame(trial_data_compile))

# print header of data compile
head(trial_data_compile)

┌ Warning: `convert(::Type{Array}, dfr::DataFrameRow)` is deprecated, use `permutedims(Vector(dfr))` instead.
│   caller = top-level scope at In[7]:11
└ @ Core ./In[7]:11


(11139, 18)(11139, 3)

┌ Warning: `head(df::AbstractDataFrame)` is deprecated, use `first(df, 6)` instead.
│   caller = top-level scope at In[7]:34
└ @ Core In[7]:34


,sub,block,reward_rate
,Int64⍰,Int64⍰,Any
1,2,1,8.16225
2,2,1,7.94251
3,2,1,7.66739
4,2,1,7.62471
5,2,1,7.89618
6,2,1,7.8407
